In [1]:
# Feature extraction
import musicbrainzngs
import ast
# Data processing
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

# Models
# from sklearn.linear_model import LinearRegression
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import BaggingRegressor

# PCA and k-fold validation
# from sklearn.decomposition import PCA
from sklearn.model_selection import KFold

# Data Processing

In [2]:
train_file = '../musicrec-data/train.csv'
test_file  = '../musicrec-data/test.csv'
prof_file = '../musicrec-data/profiles.csv'
ktrain_file = '../musicrec-data/ktrain.csv'
kvalid_file = '../musicrec-data/kvalid.csv'
# change this to reflect different solutions
# soln_file  = 'global_median.csv'

In [18]:
# Split train into 4/5 ktrain, 1/5 kvalid
train_df = pd.read_csv(train_file)
msk = np.random.rand(len(train_df)) < 0.8
ktrain = train_df[msk]
kvalid = train_df[~msk]
ktrain.to_csv(ktrain_file)
kvalid.to_csv(kvalid_file)

In [6]:
# Load the ktrain data
ktrain_data = {}
with open(ktrain_file, 'r') as ktrain_fh:
    ktrain_csv = csv.reader(ktrain_fh, delimiter=',', quotechar='"')
    next(ktrain_csv, None)
    for row in ktrain_csv:
        user   = row[1]
        artist = row[2]
        plays  = int(row[3])
    
        if not user in ktrain_data:
            ktrain_data[user] = {}
        
        ktrain_data[user][artist] = plays

In [109]:
# Load the full training data.
train_data = {}
with open(train_file, 'r') as train_fh:
    train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
    next(train_csv, None)
    for row in train_csv:
        user   = row[0]
        artist = row[1]
        plays  = int(row[2])
    
        if not user in train_data:
            train_data[user] = {}
        
        train_data[user][artist] = plays

In [5]:
# Load the test data (with all plays 0)
test_data = {}
with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)
    for row in test_csv:
        user   = row[1]
        artist = row[2]
    
        if not user in test_data:
            test_data[user] = {}
        
        test_data[user][artist] = 0

In [9]:
# ktrain_file = '../musicrec-data/ktrain.csv'
# kvalid_file = '../musicrec-data/kval.csv'
# # Separate full train_file into training set and validation set (for kfold)
# c = 0
# with open(train_file, 'r') as fulltrain_fh:
#     fulltrain_csv = csv.reader(fulltrain_fh, delimiter=',', quotechar='"')
#     next(fulltrain_csv, None)
#     rowsoftrain = len(list(fulltrain_csv))
#     print rowsoftrain

#     with open(ktrain_file, 'w') as ktrain_fh:
#         ktrain_csv = csv.writer(ktrain_fh,
#                               delimiter=',',
#                               quotechar='"',
#                               quoting=csv.QUOTE_MINIMAL)
        
#         with open(kvalid_file, 'w') as kvalid_fh:
#             kvalid_csv = csv.writer(kvalid_fh,
#                                    delimiter=',',
#                                    quotechar='"',
#                                    quoting=csv.QUOTE_MINIMAL)
            
#             for row in fulltrain_csv:
# #                 user   = row[0]
# #                 artist = row[1]
# #                 plays  = row[2]
# #                 print(user, artist, plays)
# #                 assert(0 == 1)
#                 print row
#                 if (c < (rowsoftrain * 4 / 5)):
#                     ktrain_csv.writerow(row)
#                 else:
#                     kvalid_csv.writerow(row)
#                 break
#                 c += 1

4154804


# Data Exploration: k-means

In [3]:
import pandas as pd
import sklearn.preprocessing as Pre
from sklearn.cluster import KMeans

# Load the training data
profiles = pd.read_csv(prof_file, sep=',')

In [4]:
null_age_ids = profiles[profiles['age'].isnull()]['user']
null_sex_ids = profiles[profiles['sex'].isnull()]['user']
null_cty_ids = profiles[profiles['country'].isnull()]['user']

# print null_age_ids
# print null_sex_ids
# print null_cty_ids # EMPTY!!!

null = null_age_ids + null_sex_ids
print null.shape
null_ids = set(null)
print len(null_ids)

(55432L,)
8946


In [7]:
# Get total user listens
user_listens_dict = {user:sum(ktrain_data[user].values()) for user in profiles['user']}

In [34]:
user_listens_df = pd.DataFrame(user_listens_dict.items(), columns=['user', 'plays'])

In [39]:
profile_augmented = profiles.merge(user_listens_df).dropna()

In [40]:
profile_augmented.head()

,user,sex,age,country,plays
0,fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,f,25.0,Sweden,1286
1,5909125332c108365a26ccf0ee62636eee08215c,m,29.0,Iceland,4683
2,d1867cbda35e0d48e9a8390d9f5e079c9d99ea96,m,30.0,United States,472
3,63268cce0d68127729890c1691f62d5be5abd87c,m,21.0,Germany,66869
4,02871cd952d607ba69b64e2e107773012c708113,m,24.0,Netherlands,2580


In [41]:
profiles_onehot.head()

,user,age,plays,sex_f,sex_m,country_Afghanistan,country_Albania,country_Algeria,country_American Samoa,country_Andorra,...,country_Vanuatu,country_Venezuela,country_Viet Nam,"country_Virgin Islands, British","country_Virgin Islands, U.s.",country_Wallis and Futuna,country_Western Sahara,country_Yemen,country_Zambia,country_Zimbabwe
0,fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,25.0,1286,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5909125332c108365a26ccf0ee62636eee08215c,29.0,4683,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,d1867cbda35e0d48e9a8390d9f5e079c9d99ea96,30.0,472,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,63268cce0d68127729890c1691f62d5be5abd87c,21.0,66869,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,02871cd952d607ba69b64e2e107773012c708113,24.0,2580,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
# Convert categorical data to one-hot
profiles_onehot = pd.get_dummies(profile_augmented, columns=['sex', 'country'])
# profiles_onehot.head()
# Standardize data
profiles_std = Pre.scale(profiles_onehot.iloc[:, 1:].values)

In [43]:
profiles_std.shape

(177854L, 242L)

In [6]:
kmeans_scores = []
k_values = [5,10,20,50,100]

for k in k_values:
    # Perform KMeans
    KMeansClassifier = KMeans(n_clusters=k, init='k-means++')
    kmeans = KMeansClassifier.fit(profiles_std)
    score = kmeans.score(profiles_std)
    kmeans_scores.append(score)
    print "K: ", k
    print "score: ", score
    
print kmeans_scores

plt.plot(k_values,kmeans_scores)
plt.show()

# from sklearn.decomposition import PCA
# centers = kmeans.cluster_centers_
# print centers
# pca = PCA(n_components=2)
# pca.fit(centers)
# print pca.explained_variance_ratio_ 
# pca.transform(centers)

MemoryError: 

In [55]:
# CLUSTER MEDIAN: k=100

# Perform KMeans
KMeansClassifier = KMeans(n_clusters=5, init='k-means++')
kmeans = KMeansClassifier.fit(profiles_std)

In [63]:
# Classify each user to its cluster
cluster_pred = dict(zip(profiles['user'].values, kmeans.predict(profiles_std)))

In [68]:
cluster_pred.values().count(4)

19

In [69]:
artist_df = pd.read_csv('../musicrec-data/artists.csv')
artists = artist_df['artist'].values

In [70]:
artist_cluster_dict = {artist:{k:[] for k in np.unique(cluster_pred.values())} for artist in artists}
for user, user_data in ktrain_data.iteritems():
    if user in cluster_pred:
        cluster = cluster_pred[user]
        for artist, plays in user_data.iteritems():
            artist_cluster_dict[artist][cluster].append(plays)

In [72]:
artist_cluster_medians = {artist:[np.median(artist_cluster_dict[artist][cluster]) for cluster in np.unique(cluster_pred.values())] for artist in artists}

In [55]:
# Take the cluster median
# If the user has no data, use the global median.

global_median = 118.0

# Compute the cluster median.
n_clusters = len(np.unique(cluster_pred.values()))
cluster_median_dict = {k:[] for k in np.unique(cluster_pred.values())}
for user, user_data in ktrain_data.iteritems():
    if user in cluster_pred:
        cluster = cluster_pred[user]
        for artist, plays in user_data.iteritems():
            cluster_median_dict[cluster].append(plays)
        # get cluster index for this user
cluster_medians = [np.median(plays) for cluster, plays in cluster_median_dict.iteritems()]

# Write out test solutions.
# with open(test_file, 'r') as test_fh:
#     test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
#     next(test_csv, None)

#     with open(soln_file, 'w') as soln_fh:
#         soln_csv = csv.writer(soln_fh,
#                               delimiter=',',
#                               quotechar='"',
#                               quoting=csv.QUOTE_MINIMAL)
#         soln_csv.writerow(['Id', 'plays'])

#         for row in test_csv:
#             id     = row[0]
#             user   = row[1]
#             artist = row[2]

#             if user in user_medians:
#                 soln_csv.writerow([id, user_medians[user]])
#             else:
#                 print "User", id, "not in training data."
#                 soln_csv.writerow([id, global_median])

In [56]:
print cluster_medians

[115.0, 117.0, 100.0, 116.0, 118.0, 113.0, 114.0, 120.0, 118.0, 117.0]


# globalmedian.py

In [ ]:
# Predict via the median number of plays.

# Compute the global median.
plays_array = []
for user, user_data in train_data.iteritems():
    for artist, plays in user_data.iteritems():
        plays_array.append(plays)
global_median = np.median(np.array(plays_array))
print "global median:", global_median

# Write out test solutions.
with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open(soln_file, 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])

        for row in test_csv:
            id     = row[0]
            user   = row[1]
            artist = row[2]

            soln_csv.writerow([id, global_median])

# usermedian.py

In [ ]:
# Compute the global median and per-user median.
plays_array  = []
user_medians = {}
for user, user_data in train_data.iteritems():
    user_plays = []
    for artist, plays in user_data.iteritems():
        plays_array.append(plays)
        user_plays.append(plays)

    user_medians[user] = np.median(np.array(user_plays))
global_median = np.median(np.array(plays_array))

In [18]:
# Predict via the user-specific median.
# If the user has no data, use the global median.
# Write out test solutions.
soln_file = 'user_median.csv'
with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open(soln_file, 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])

        for row in test_csv:
            id     = row[0]
            user   = row[1]
            artist = row[2]

            if user in user_medians:
                soln_csv.writerow([id, user_medians[user]])
            else:
                print "User", id, "not in training data."
                soln_csv.writerow([id, global_median])

# Feature Extraction

In [7]:
artist_genre_csv = pd.read_csv('genre_dicts/artist_genre_dict.csv', header=None)

In [8]:
artist_genre_dict = {row[0]:row[1] for ind, row in artist_genre_csv.iterrows()}

In [54]:
user_genre_csv = pd.read_csv('genre_dicts/user_genre_dict.csv', header=None)

In [55]:
user_genre_dict = {row[0]:ast.literal_eval(row[1]) for ind, row in user_genre_csv.iterrows()}

In [ ]:
k_user_genre_csv = pd.read_csv('genre_dicts/k_user_genre_dict.csv', header=None)

In [ ]:
k_user_genre_dict = {row[0]:ast.literal_eval(row[1]) for ind, row in k_user_genre_csv.iterrows()}

In [27]:
# set user agent before any requests are made
musicbrainzngs.set_useragent(2,1)

In [28]:
# function for getting the mode tag for an artist (alphabetical order)
def mode_tag(artist_id):
    result = musicbrainzngs.get_artist_by_id(artist_id,includes=['tags'])
    tags = result['artist']['tag-list']
    mode_tag = None
    mode = None
    for tag_dict in tags:
        if tag_dict['count'] > mode:
            mode = tag_dict['count']
            mode_tag = tag_dict['name']
    return mode_tag

# mode_tag('a74b1b7f-71a5-4011-9441-d0b5e4122711')

In [29]:
# read in artist information
artists = pd.read_csv('../musicrec-data/artists.csv', sep=',')
artist_ids = artists['artist'].values

In [31]:
def build_artist_genre_dict(artists):
    artist_genre_dict = {}
    for artist_id in artists:
        print num
        # update artist_genre dict
        try: 
            genre = mode_tag(artist_id)
            artist_genre_dict[artist_id] = genre
        except (KeyError, musicbrainzngs.ResponseError):
            artist_genre_dict[artist_id] = None
    return artist_genre_dict
        

In [ ]:
artist_genre_dict = build_artist_genre_dict(artist_ids)

In [82]:
for k,v in artist_genre_dict.iteritems():
    print k
    print v
    break

print len(artist_genre_dict)

03098741-08b3-4dd7-b3f6-1b0bfa2c879c
alternative rock
2000


In [35]:
for (k, v) in user_genre_dict.iteritems():
    print k
    print v['folk rock']
    break

fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d
(237, 1)


In [18]:
for (k, v) in artist_dict.iteritems():
    print k
    print v
    break

alternative rock
0                  american
1            rock and indie
2          canadian pianist
3            rock and indie
4                      rock
5                       pop
6               black metal
7                 dance-pop
8          industrial metal
9                    grunge
10               electronic
11         melodic hardcore
12                indie pop
13                      ska
14                      NaN
15                 austrian
16                 american
17                    irish
18               do not use
19                     punk
20          soul and reggae
21        alternative metal
22                 american
23                   celtic
24         crossover thrash
25              alternative
26                   trance
27                 american
28             country rock
29                   french
               ...         
1969              folk rock
1970         rock and indie
1971                    NaN
1972             electronic
197

In [80]:
# SAVE TO CSV
with open('artist_genre_dict.csv', 'wb') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in artist_genre_dict.items():
        if value:
            writer.writerow([key, value.encode('ascii', 'ignore').decode('ascii')])
        else:
            writer.writerow([key, value])

In [9]:
def build_user_genre_dict(train_data):
    user_genre_dict = {}
    num = 1
    for user, artist_info in train_data.iteritems():
        if num % 10000 == 0:
            print num
        genre_dict = {}
        for artist, plays in artist_info.iteritems():
            genre = artist_genre_dict[artist]
            # only proceed if genre is not None
            if genre: 
                # if the user hasn't already listened to the genre
                if genre not in genre_dict:
                    genre_dict[genre] = (plays, 1)
                # otherwise, add number of plays to current total
                else:
                    (num_genre_plays, num_genre_artists) = genre_dict[genre]
                    genre_dict[genre] = (num_genre_plays + plays, num_genre_artists + 1)
        user_genre_dict[user] = genre_dict
        num += 1
    return user_genre_dict


In [10]:
k_user_genre_dict = build_user_genre_dict(ktrain_data)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000


In [83]:
for k,v in user_genre_dict.iteritems():
    print k
    print v
    break
    
print len(user_genre_dict)

fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d
{'folk rock': (237, 1), 'alternative rock': (222, 3), 'finland': (66, 1), 'reggae': (70, 1), 'indie folk': (136, 1), 'pop': (169, 3), 'pop punk': (47, 1), 'british': (351, 3), 'rock': (84, 1), 'indie pop': (72, 1), 'norway': (40, 1), 'folk': (32, 1)}
233286


In [ ]:
# SAVE AS CSV
# with open('user_genre_dict.csv', 'wb') as csv_file:
#     writer = csv.writer(csv_file)
#     for key, value in user_genre_dict.items():
#        writer.writerow([key, value])

In [12]:
# SAVE AS CSV
with open('k_user_genre_dict.csv', 'wb') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in k_user_genre_dict.items():
       writer.writerow([key, value])

## Artist information, searching, and browsing

In [6]:
# artist info: year, genre (tag), area, ratings
# user info: age, gender, country

# musicbrainzngs.search_artists("radiohead")
# #musicbrainzngs.search_recordings("bennie and the jets")

# radiohead_id = "a74b1b7f-71a5-4011-9441-d0b5e4122711"
# radiohead_result = musicbrainzngs.get_artist_by_id(radiohead_id,includes=["tags"])#"releases","ratings","tags"])
# radiohead_artist = radiohead_result["artist"]

# print radiohead_artist

# print("name:\t\t%s" % radiohead_artist["name"])
# print("rating:\t\t%s" % radiohead_artist["rating"])
# print("release 0:\t%s" % radiohead_artist["release-list"][0])

# dictionary of recordings for radiohead
# musicbrainzngs.browse_recordings(radiohead_id)

# "artist" can include the following information:
# recordings, releases, release-groups, works, various-artists,
# discids, media, isrcs, aliases, annotation, area-rels, artist-rels,
# label-rels, place-rels, event-rels, recording-rels, release-rels,
# release-group-rels, series-rels, url-rels, work-rels, instrument-rels,
# tags, user-tags, ratings, user-ratings

{'name': 'Radiohead', 'area': {'sort-name': 'United Kingdom', 'iso-3166-1-code-list': ['GB'], 'id': '8a754a16-0027-3a29-b6d7-2b40ea0481ed', 'name': 'United Kingdom'}, 'country': 'GB', 'life-span': {'begin': '1991'}, 'tag-list': [{'count': '7', 'name': 'alternative rock'}, {'count': '1', 'name': 'art pop'}, {'count': '5', 'name': 'art rock'}, {'count': '0', 'name': 'bootleg'}, {'count': '0', 'name': 'britannique'}, {'count': '3', 'name': 'british'}, {'count': '2', 'name': 'britpop'}, {'count': '-1', 'name': u'c\u2019\xe9tait mieux avant'}, {'count': '4', 'name': 'electronic'}, {'count': '0', 'name': 'england'}, {'count': '1', 'name': 'english'}, {'count': '2', 'name': 'experimental rock'}, {'count': '1', 'name': 'grunge'}, {'count': '0', 'name': 'melancholic'}, {'count': '0', 'name': 'nude'}, {'count': '1', 'name': 'oxford'}, {'count': '0', 'name': 'parlophone'}, {'count': '7', 'name': 'rock'}, {'count': '0', 'name': 'rock and indie'}, {'count': '-1', 'name': 'sacred cows'}, {'count': '

In [65]:
artist_id = "f1a95c6b-fb2a-41a6-bfcb-2453fee2a38c"
result = musicbrainzngs.get_artist_by_id(artist_id,includes=["tags"])#"releases","ratings","tags"])
artist = result["artist"]

print artist


ResponseError: caused by: HTTP Error 404: Not Found

## Cover art information

In [42]:
# gets the list of cover art associated with a release
musicbrainzngs.get_image_list("081e7033-5282-4bd7-9963-e64edf8c693a")

# downloads the front cover art of a release
# musicbrainzngs.get_image_front("081e7033-5282-4bd7-9963-e64edf8c693a")

{u'images': [{u'approved': True,
   u'back': False,
   u'comment': u'',
   u'edit': 35299266,
   u'front': True,
   u'id': 11628918400,
   u'image': u'http://coverartarchive.org/release/081e7033-5282-4bd7-9963-e64edf8c693a/11628918400.jpg',
   u'thumbnails': {u'large': u'http://coverartarchive.org/release/081e7033-5282-4bd7-9963-e64edf8c693a/11628918400-500.jpg',
    u'small': u'http://coverartarchive.org/release/081e7033-5282-4bd7-9963-e64edf8c693a/11628918400-250.jpg'},
   u'types': [u'Front']}],
 u'release': u'https://musicbrainz.org/release/081e7033-5282-4bd7-9963-e64edf8c693a'}

# Training

In [12]:
# Function for k-fold cross validation using MAE (mean absolute error)
def kfold(k, model, data):
    kf = KFold(n_splits=k)
    maes = []
    for train_fold_index, validate_fold_index in kf.split(data):
        train_fold = data[train_fold_index]
        test_fold = data[validate_fold_index]
        X_train_fold = train_fold[:, :-1]
        Y_train_fold = train_fold[:, -1]
        X_test_fold = test_fold[:, :-1]
        Y_test_fold = test_fold[:, -1]
        model.fit(X_train_fold, Y_train_fold)
        Y_hat = model.predict(X_test_fold)
        mae = np.mean([abs(Y_hat[i] - Y_test_fold[i]) for i in range(len(Y_test_fold))])
        maes.append(mae)
        print "MAEs: ", maes
    return np.mean(maes)

# Making Predictions

In [76]:
# Compute the global median and per-user median.
plays_array  = []
user_medians = {}
for user, user_data in ktrain_data.iteritems():
    user_plays = []
    for artist, plays in user_data.iteritems():
        plays_array.append(plays)
        user_plays.append(plays)
    user_medians[user] = np.median(np.array(user_plays))
global_median = np.median(np.array(plays_array))

In [110]:
# Compute the global median and per-user median.
plays_array  = []
user_medians_2 = {}
for user, user_data in train_data.iteritems():
    user_plays = []
    for artist, plays in user_data.iteritems():
        plays_array.append(plays)
        user_plays.append(plays)
    user_plays = np.array(user_plays)
    user_medians_2[user] = np.median(user_plays[user_plays <= 10000])
global_median = np.median(np.array(plays_array))

In [81]:
# Compute artist medians
artist_dict = {}
for user, user_data in ktrain_data.iteritems():
    for artist, plays in user_data.iteritems():
        if artist not in artist_dict:
            artist_dict[artist] = [plays]
        else:
            artist_dict[artist].append(plays)

In [82]:
artist_medians = {artist:np.median(plays) for artist, plays in artist_dict.iteritems()}

In [83]:
# Intended for use with kvalid.csv or test.csv as predict_file
def make_predictions(predict_file, alpha, user_genre_dict, artist_genre_dict, global_median, user_medians, test_flag=False, soln_file='dummy.csv'):
    with open(predict_file, 'r') as predict_fh:
        predict_csv = csv.reader(predict_fh, delimiter=',', quotechar='"')
        next(predict_csv, None)
        
        with open(soln_file, 'w') as soln_fh:
            soln_csv = csv.writer(soln_fh,
                                  delimiter=',',
                                  quotechar='"',
                                  quoting=csv.QUOTE_MINIMAL)
            soln_csv.writerow(['Id', 'plays'])
            predictions = {} # dictionary of id -> prediction
            for row in predict_csv:
                id     = row[0]
                user   = row[1]
                artist = row[2]
                
#                 genre = artist_genre_dict[artist]
#                 try:
#                     user_genre_avg_plays = user_genre_dict[user][genre][0] / user_genre_dict[user][genre][1]
#                 except KeyError:
#                     genre = None

                # If artist has a genre
#                 if genre:
#                     if user in user_medians:
#                         predictions[id] = alpha * (user_genre_avg_plays) + (1. - alpha) * user_medians[user]
#                     else:
#                         predictions[id] = alpha * (user_genre_avg_plays) + (1. - alpha) * global_median
#                 else:
#                     if user in user_medians:
#                         predictions[id] = user_medians[user]
#                     else:
#                         predictions[id] = global_median

#                 if user in user_medians:
#                     predictions[id] = alpha * artist_medians[artist] + (1. - alpha) * user_medians[user]
#                 else:
#                     predictions[id] = alpha * artist_medians[artist] + (1. - alpha) * global_median

#                 if user in cluster_pred:
#                     predictions[id] = alpha * cluster_medians[cluster_pred[user]] + (1. - alpha) * user_medians[user]
#                 else:
#                     if user in user_medians:
#                         predictions[id] = user_medians[user]
#                     else:
#                         predictions[id] = global_median
                
                if user in cluster_pred and not np.isnan(artist_cluster_medians[artist][cluster_pred[user]]):
                    predictions[id] = alpha * artist_cluster_medians[artist][cluster_pred[user]] + (1. - alpha) * user_medians[user]
                else:
                    if user in user_medians:
                        predictions[id] = alpha * artist_medians[artist] + (1 - alpha) * user_medians[user]
                    else:
                        predictions[id] = alpha * artist_medians[artist] + (1 - alpha) * global_median
                    
                # If test flag, write prediction to soln_csv
                if test_flag:
                    soln_csv.writerow([id, predictions[id]])
            return predictions
            

In [79]:
# Compute the global median and per-user median.
# plays_array  = []
# user_medians = {}
# for user, user_data in train_data.iteritems():
#     user_plays = []
#     for artist, plays in user_data.iteritems():
#         plays_array.append(plays)
#         user_plays.append(plays)
#     user_medians[user] = np.median(np.array(user_plays))
# global_median = np.median(np.array(plays_array))

In [74]:
# Validate data
kvalidate = pd.read_csv(kvalid_file)

In [108]:
# Tune prediction
alphas = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.]
for alpha in alphas:
    preds = make_predictions(kvalid_file, alpha, {}, {}, global_median, user_medians_2)
    ordered_preds = np.array([preds[str(i)] for i in kvalidate.iloc[:,0].values])
    print 'alpha:', alpha, np.mean(abs(ordered_preds-kvalidate['plays'].values))

alpha: 0 138.171613956
alpha: 0.1 138.110371254
alpha: 0.2 139.689028463
alpha: 0.3 142.739746876
alpha: 0.4 147.251354803
alpha: 0.5 153.19799599
alpha: 0.6 160.478133233
alpha: 0.7 168.891355874
alpha: 0.8 178.182567199
alpha: 0.9 188.104051169
alpha: 1.0 198.463394511


In [17]:
kvalidate.head()

,Unnamed: 0,user,artist,plays
0,4,b85fcaef67d2669cd99b334b5e8c8705263db2cf,a3cb23fc-acd3-4ce0-8f36-1e5aa6a18432,220
1,8,9f748976d303db79f61bf570d9549d6335b11b0e,2fddb92d-24b2-46a5-bf28-3aed46f4684c,705
2,17,d7cf77cd6288e017a6c3c5fd5a18f8e57ddfff9d,fc01cdec-f2e3-4d43-a1f7-7945ca43618c,25
3,18,98854e7fae03e5d02637f25c5940a9e6e1042bad,1c89bbcc-a253-452a-9181-546eb536e06f,7
4,21,6afe9bb3d915f7f9d7778858d468e2102f29819a,39a1b293-8e17-4a1c-b50b-034c40e24461,11


In [114]:
# Test prediction
make_predictions(test_file, 0.1, None, None, global_median, user_medians_2, True)

{'287144': 651.15000000000009,
 '287145': 358.69999999999999,
 '287146': 1429.2,
 '287147': 2044.6000000000001,
 '287140': 184.40000000000001,
 '287141': 309.39999999999998,
 '287142': 422.5,
 '287143': 38.600000000000001,
 '1307837': 48.700000000000003,
 '287148': 98.100000000000009,
 '287149': 46.500000000000007,
 '378468': 95.900000000000006,
 '378469': 95.849999999999994,
 '4125744': 136.25,
 '378465': 81.800000000000011,
 '2008919': 22.600000000000001,
 '2008918': 314.69999999999999,
 '2008917': 164.80000000000001,
 '2008916': 229.40000000000001,
 '2008915': 129.59999999999999,
 '2008914': 106.60000000000001,
 '2008913': 38.0,
 '2008912': 54.100000000000001,
 '2008911': 90.300000000000011,
 '378463': 130.0,
 '899880': 52.700000000000003,
 '378460': 1112.05,
 '378461': 421.19999999999999,
 '2577849': 99.150000000000006,
 '2843839': 125.59999999999999,
 '2843838': 444.34999999999997,
 '2871425': 71.200000000000003,
 '2871424': 254.19999999999999,
 '2871423': 53.800000000000004,
 '28

In [16]:
for (k, v) in yhat.iteritems():
    print k
    print v
    break

287144
712.0


In [ ]:
# Predict via the user-genre median.
# If the user has no data for a particular genre, use the user median.
# If the user has no data, use the global median.

# Write out test solutions.
with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open(soln_file, 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])

        for row in test_csv:
            id     = row[0]
            user   = row[1]
            artist = row[2]

            if user in user_medians:
                soln_csv.writerow([id, user_medians[user]])
            else:
                print "User", id, "not in training data."
                soln_csv.writerow([id, global_median])